In [ ]:
from pathlib import Path
import os
name = 
ROOT_DIR = Path("./"+name)
# ROOT_DIR = Path("./Kitten")

In [ ]:
import trimesh
import numpy as np
from scipy.spatial import cKDTree
from pathlib import Path

mesh = trimesh.load_mesh(ROOT_DIR/(name+".obj"), process=False)
vertices = mesh.vertices
faces = mesh.faces

tree = cKDTree(vertices)

merge_threshold = 1e-6
_, idx_map = tree.query(vertices, k=1, distance_upper_bound=merge_threshold)

idx_map[idx_map >= len(vertices)] = np.arange(len(vertices))[idx_map >= len(vertices)]

new_faces = np.array([[idx_map[v] for v in face] for face in faces])

new_mesh = trimesh.Trimesh(vertices=vertices, faces=new_faces, process=True)

output_path = ROOT_DIR/"final.obj"
new_mesh.export(output_path)

output_path


In [ ]:
patch_num = 0

def split_obj_by_o_or_g(input_obj_path, output_dir):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    with open(input_obj_path, 'r') as f:
        lines = f.readlines()

    vertex_lines = [line for line in lines if line.startswith('v')]
    header_lines = [line for line in lines if not line.startswith(('v', 'f', 'o', 'g'))]

    objects = []
    current_name = None
    current_faces = []

    for line in lines:
        if line.startswith('o ') or line.startswith('g '):
            if current_name and current_faces:
                objects.append((current_name, current_faces))
                current_faces = []
            current_name = line.strip().split(' ', 1)[-1]
        elif line.startswith('f '):
            current_faces.append(line)

    if current_name and current_faces:
        objects.append((current_name, current_faces))

    print(f"Detected {len(objects)} patches")

    for i, (name, faces) in enumerate(objects):
        filename = f"patch_{i}.obj"
        with open(os.path.join(output_dir, filename), 'w') as f:
            f.writelines(header_lines)
            f.writelines(vertex_lines)
            f.write(f"o {name}\n")  # 
            f.writelines(faces)
        print(f"Save patch_{i}: {name} to {filename}")
    return len(objects)

patch_num = split_obj_by_o_or_g(ROOT_DIR / (name+"_cut.obj"), ROOT_DIR / "patches")


In [ ]:
import os
import numpy as np
from tqdm import tqdm

def load_obj(filepath):
    vertices = []
    faces = []
    with open(filepath, 'r') as f:
        for line in f:
            if line.startswith('v '):
                vertices.append(list(map(float, line.strip().split()[1:4])))
            elif line.startswith('f '):
                face = []
                for v in line.strip().split()[1:]:
                    face.append(int(v.split('/')[0]) - 1)
                faces.append(face)
    return np.array(vertices), np.array(faces)

def face_vertices(vertices, face):
    return np.array([vertices[idx] for idx in face])

def are_faces_equal(f1, f2, tol=1e-6):
    for p in f1:
        if not np.any(np.linalg.norm(f2 - p, axis=1) < tol):
            return False
    return True

def build_seg_from_geometry(assembled_path, patch_dir, patch_ids, output_path='seg.txt', tol=1e-6):
    print("Loading assembled mesh...")
    v_all, f_all = load_obj(assembled_path)
    f_all_coords = [face_vertices(v_all, face) for face in f_all]

    face_to_patch = [-1] * len(f_all)

    print("Processing patches...")
    for pid in patch_ids:
        patch_path = os.path.join(patch_dir, f'patch_{pid}.obj')
        v_patch, f_patch = load_obj(patch_path)
        for face in tqdm(f_patch, desc=f"🔧 Matching patch_{pid}"):
            coords_patch = face_vertices(v_patch, face)

            for idx, coords_asm in enumerate(f_all_coords):
                if face_to_patch[idx] == -1 and are_faces_equal(coords_patch, coords_asm, tol=tol):
                    face_to_patch[idx] = pid
                    break
            else:
                print(f"Patch {pid} face not matched: {face}")

    unmatched = sum(1 for x in face_to_patch if x == -1)
    print(f"Matching complete. Unmatched faces: {unmatched}/{len(f_all)}")

    with open(output_path, 'w') as f:
        for pid in face_to_patch:
            f.write(f"{pid}\n")

    print(f"seg.txt written to: {ROOT_DIR / output_path}")


# build_seg_from_geometry("Bear.obj", "./patches", range(8), output_path="Bear_seg.txt", tol=1e-6)


In [ ]:
def build_seg_from_geometry(assembled_path, patch_dir, patch_ids, output_path='seg.txt', tol=1e-6):
    print("Loading assembled mesh...")
    v_all, f_all = load_obj(assembled_path)

    f_all_coords = [face_vertices(v_all, face) for face in f_all]
    f_all_centers = [np.mean(coords, axis=0) for coords in f_all_coords]
    
    face_to_patch = [-1] * len(f_all)
    unmatched_faces = set(range(len(f_all)))

    print("Processing patches...")
    for pid in patch_ids:
        patch_path = os.path.join(patch_dir, f'patch_{pid}.obj')
        if not os.path.exists(patch_path):
            print(f"Warning: Patch {pid} not found at {patch_path}")
            continue
            
        v_patch, f_patch = load_obj(patch_path)

        patch_centers = []
        for face in f_patch:
            coords = face_vertices(v_patch, face)
            patch_centers.append(np.mean(coords, axis=0))
        
        for patch_idx, face in enumerate(tqdm(f_patch, desc=f"🔧 Matching patch_{pid}")):
            coords_patch = face_vertices(v_patch, face)
            patch_center = patch_centers[patch_idx]

            candidate_indices = []
            for asm_idx in unmatched_faces:
                if np.linalg.norm(patch_center - f_all_centers[asm_idx]) < tol * 10:
                    candidate_indices.append(asm_idx)

            for asm_idx in candidate_indices:
                if are_faces_equal(coords_patch, f_all_coords[asm_idx], tol=tol):
                    face_to_patch[asm_idx] = pid
                    unmatched_faces.remove(asm_idx)
                    break
            else:
                print(f"Patch {pid} face {patch_idx} not matched")

    unmatched = len(unmatched_faces)
    print(f"Matching complete. Unmatched faces: {unmatched}/{len(f_all)}")

    with open(output_path, 'w') as f:
        for pid in face_to_patch:
            f.write(f"{pid}\n")

    print(f"seg.txt written to: {output_path}")
    return face_to_patch

In [ ]:
build_seg_from_geometry(ROOT_DIR/(name+".obj"), ROOT_DIR/"patches", range(patch_num), output_path=ROOT_DIR/"final_seg.txt", tol=1e-5)